In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import mathis as mt
import math
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv("datasets/modified_train.csv")
test = pd.read_csv("datasets/modified_test.csv")

train = train.drop(columns = ['Unnamed: 0'])
test = test.drop(columns = ['Unnamed: 0'])

house = train.append(test)

# Data Engineering

***Determine features to bucket together***

In [3]:
candidates = mt.feature_bucket_candidates(house[house['training'] == 1])
buckets = mt.greedy_bucket_selection(candidates)

In [4]:
all_dummies = pd.get_dummies(train, columns = buckets.keys(), drop_first = True, dummy_na = True)
mt.corrs_selection(all_dummies, 'sale_price')

,year_built,year_remod/add,mas_vnr_area,total_bsmt_sf,1st_flr_sf,gr_liv_area,garage_yr_blt,garage_area,sale_price,foundation_PConc,garage_cars_3.0
0,0.571849,0.55037,0.503579,0.629303,0.618486,0.697038,0.556146,0.649897,1.0,0.529047,0.61911


In [5]:
house = mt.map_buckets(house, buckets)

***Dummy categoricals***

In [6]:
house.shape

(2929, 82)

In [7]:
house = pd.get_dummies(house, columns = buckets.keys(), drop_first = True, dummy_na = True)

In [8]:
house.shape

(2929, 222)

In [9]:
train = house[house['training'] == 1]
test = house[house['training'] == 0]

In [10]:
x_list = house.columns.tolist()
x_list.remove('sale_price')
X = train[x_list]
y = train['sale_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)
house_lasso = LassoCV()

In [11]:
mt.corrs_selection(train, 'sale_price')

,year_built,year_remod/add,mas_vnr_area,total_bsmt_sf,1st_flr_sf,gr_liv_area,garage_yr_blt,garage_area,sale_price,foundation_PConc
0,0.571849,0.55037,0.503579,0.629303,0.618486,0.697038,0.556146,0.649897,1.0,0.529047


In [12]:
house_lasso.fit(X_train, y_train)

LassoCV()

In [13]:
house_lasso.alphas_

array([3.93920134e+12, 3.67371236e+12, 3.42611644e+12, 3.19520766e+12,
       2.97986136e+12, 2.77902867e+12, 2.59173144e+12, 2.41705742e+12,
       2.25415584e+12, 2.10223328e+12, 1.96054979e+12, 1.82841530e+12,
       1.70518623e+12, 1.59026238e+12, 1.48308402e+12, 1.38312912e+12,
       1.28991085e+12, 1.20297517e+12, 1.12189867e+12, 1.04628646e+12,
       9.75770251e+11, 9.10006602e+11, 8.48675203e+11, 7.91477334e+11,
       7.38134411e+11, 6.88386622e+11, 6.41991668e+11, 5.98723578e+11,
       5.58371613e+11, 5.20739235e+11, 4.85643153e+11, 4.52912430e+11,
       4.22387648e+11, 3.93920134e+11, 3.67371236e+11, 3.42611644e+11,
       3.19520766e+11, 2.97986136e+11, 2.77902867e+11, 2.59173144e+11,
       2.41705742e+11, 2.25415584e+11, 2.10223328e+11, 1.96054979e+11,
       1.82841530e+11, 1.70518623e+11, 1.59026238e+11, 1.48308402e+11,
       1.38312912e+11, 1.28991085e+11, 1.20297517e+11, 1.12189867e+11,
       1.04628646e+11, 9.75770251e+10, 9.10006602e+10, 8.48675203e+10,
      